# Final Project
## Nicholas Schenone - A13599911

- 3 trials
- 7 classifiers
    - SVM
    - Logistic Regression
    - Decision Tree
    - Perceptron
    - Multilayer Perceptron
    - KNN
    - Random Forest
- 3 datasets
    - Heart Disease: https://www.kaggle.com/ronitf/heart-disease-uci
    - Mushroom: https://archive.ics.uci.edu/ml/datasets/Mushroom
    - Adult Data Set: https://archive.ics.uci.edu/ml/datasets/Adult
- 3 partitions (20/80, 50/50, 80/20)
- 3 accuracies per (train, validation, test)

### Imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

import json

import seaborn as sns

### Pre-Process Data

In [2]:
def adult_pre_process(data_path="data/adult/adult.csv", split=0.2):
    df_adult = pd.read_csv(data_path)
    df_adult_one_hot = pd.get_dummies(df_adult);
    
    X = df_adult_one_hot.iloc[:,0 : len(df_adult_one_hot.columns) - 1]
    X = StandardScaler().fit_transform(X)

    y = df_adult_one_hot.iloc[:, len(df_adult_one_hot.columns) - 1]
    y = y.values.ravel()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split)

    return X, y, X_train, X_test, y_train, y_test

def heart_pre_process(data_path="data/heart_disease/heart.csv", split=0.2):
    df_heart = pd.read_csv(data_path)
    X = df_heart.iloc[:, 0 : len(df_heart.columns) - 1]
    X = StandardScaler().fit_transform(X)

    y = df_heart.iloc[:, len(df_heart.columns) - 1]
    y = y.values.ravel()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split)

    return X, y, X_train, X_test, y_train, y_test

def mushroom_pre_process(data_path="data/mushroom/mushroom.csv", split=0.2):
    df_mushroom = pd.read_csv(data_path, header=None)
    df_mush_one_hot = pd.get_dummies(df_mushroom);
    
    X = df_mush_one_hot.iloc[:,1:]
    X = StandardScaler().fit_transform(X)

    y = df_mush_one_hot.iloc[:, :1]
    y = y.values.ravel()
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split)

    return X, y, X_train, X_test, y_train, y_test

def pre_process(dataset, split=0.2):
    if dataset == "happy":
        return happiness_pre_process(split=split)
    elif dataset == "mush":
        return mushroom_pre_process(split=split)
    elif dataset == "heart":
        return heart_pre_process(split=split)
    elif dataset == "adult":
        return adult_pre_process(split=split)

In [3]:
heart_X, heart_y, heart_X_train, heart_X_test, heart_y_train, heart_y_test = heart_pre_process(split=0.2)

mush_X, mush_y, mush_X_train, mush_X_test, mush_y_train, mush_y_test = mushroom_pre_process(split=0.2)

adult_X, adult_y, adult_X_train, adult_X_test, adult_y_train, adult_y_test = adult_pre_process(split=0.5)

### Classifiers and Functions

In [4]:
# SVM
def clf_SVM(param_grid):
    return svm.SVC(C = param_grid["C"],
                   gamma=param_grid["gamma"],
                   kernel=param_grid["kernel"],
                   max_iter = 10000)

# Logistic Regression
def clf_log(param_grid):
    return LogisticRegression(C = param_grid["C"],
                              penalty = param_grid["penalty"],
                              solver="liblinear",
                              max_iter = 10000)

# Decision Tree
def clf_tree(param_grid):
    return DecisionTreeClassifier(criterion=param_grid["criterion"],
                                  max_depth=param_grid["max_depth"])

# Perceptron
def clf_perc(param_grid):
    return Perceptron(penalty=param_grid["penalty"],
                      alpha=param_grid["alpha"],
                      max_iter=param_grid["max_iter"],
                      tol=param_grid["tol"],
                      early_stopping=param_grid["early_stopping"])

# Multi-Layer Perceptron
def clf_mlp(param_grid):
    return MLPClassifier(activation=param_grid["activation"],
                      solver=param_grid["solver"],
                      hidden_layer_sizes=param_grid["hidden_layer_sizes"],
                      max_iter=param_grid["max_iter"],
                      tol=param_grid["tol"],
                      early_stopping=param_grid["early_stopping"])

# KNN
def clf_knn(param_grid):
    return KNeighborsClassifier(n_neighbors=param_grid["n_neighbors"])

# Random Forest
def clf_rf(param_grid):
    return RandomForestClassifier(bootstrap=param_grid["bootstrap"],
                                 max_depth=param_grid["max_depth"],
                                 max_features=param_grid["max_features"],
                                 min_samples_leaf=param_grid["min_samples_leaf"],
                                 min_samples_split=param_grid["min_samples_split"],
                                 n_estimators=param_grid["n_estimators"])

# General
def clf(model, param_grid):
    if model == "svm":
        return clf_SVM(param_grid)
    elif model=="log":
        return clf_log(param_grid)
    elif model=="tree":
        return clf_tree(param_grid)
    elif model=="perc":
        return clf_perc(param_grid)
    elif model=="mlp":
        return clf_mlp(param_grid)
    elif model=="knn":
        return clf_knn(param_grid)
    elif model=="rf":
        return clf_rf(param_grid)
    
def train_model(classifier, X_train, y_train):
    classifier.fit(X_train, y_train)

def hyper_tune(X_train, y_train, estimator, param_grid, k_top=3):
    grid_search = RandomizedSearchCV(estimator=estimator, param_distributions=param_grid, cv=10, n_iter=20, n_jobs=-1, verbose=10)
    grid_search.fit(X_train, y_train)
    results = pd.DataFrame(grid_search.cv_results_)
    results.sort_values(by='rank_test_score', inplace=True)
    out = []
    [out.append(results.loc[i, 'params']) for i in range(k_top)]
    print(f"Best {k_top} params:", out)
    return out

def evalModel(classifer, X_test, y_test):
    y_pred = classifier.predict(X_test)
    
    accuracy= accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average="macro")
    recall = recall_score(y_test, y_pred, average="macro")
    f_score = f1_score(y_test, y_pred, average="macro") 
    
    return (accuracy, precision, recall, f_score)

## Hyperparameter Tuning

### SVM

In [8]:
svm_param_grid = {
    "C" : [1, 10, 100, 1000, 10000],
    "gamma" : [1e-6, 1e-5, 1e-4, 1e-3, 1e-2],
    "kernel" : ["linear", "rbf"]
}

In [9]:
# Mushroom SVM Tuning
best_param_grid_mush = hyper_tune(mush_X_train, mush_y_train, svm.SVC(), svm_param_grid)

with open('params/svm/best_param_grid_mush', 'w') as f:
    json.dump(best_param_grid_mush, f)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   34.7s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   36.0s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   39.3s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   57.3s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:  2

Best 3 params: [{'kernel': 'rbf', 'gamma': 1e-06, 'C': 10}, {'kernel': 'linear', 'gamma': 1e-06, 'C': 10}, {'kernel': 'rbf', 'gamma': 0.01, 'C': 10000}]


[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  2.8min finished


In [10]:
# Heart SVM Tuning
best_param_grid_heart = hyper_tune(heart_X_train, heart_y_train, svm.SVC(), svm_param_grid)

with open('params/svm/best_param_grid_heart', 'w') as f:
    json.dump(best_param_grid_heart, f)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1900s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0326s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0971s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1945s.) Setting batch_size=16.
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 131 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 151 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done 171 t

Best 3 params: [{'kernel': 'rbf', 'gamma': 0.01, 'C': 10000}, {'kernel': 'rbf', 'gamma': 1e-05, 'C': 1000}, {'kernel': 'linear', 'gamma': 0.001, 'C': 1}]


[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  2.7min finished
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [11]:
# Adult SVM Tuning
best_param_grid_adult = hyper_tune(adult_X_train, adult_y_train, svm.SVC(), svm_param_grid)

with open('params/svm/best_param_grid_adult', 'w') as f:
    json.dump(best_param_grid_adult, f)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:  4

Best 3 params: [{'kernel': 'rbf', 'gamma': 1e-06, 'C': 1000}, {'kernel': 'linear', 'gamma': 0.0001, 'C': 1}, {'kernel': 'rbf', 'gamma': 0.001, 'C': 100}]


### Logistic Regression

In [5]:
log_param_grid = {
    "C" : [1, 10, 100, 1000, 10000],
    "penalty" : ["l1", "l2"],
}

In [6]:
# Mushroom Logistic Regression Tuning
best_param_grid_mush = hyper_tune(mush_X_train, mush_y_train, LogisticRegression(), log_param_grid)

with open('params/log/best_param_grid_mush', 'w') as f:
    json.dump(best_param_grid_mush, f)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 10 is smaller than n_iter=20. Running 10 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:    8.2s


Best 3 params: [{'penalty': 'l1', 'C': 1}, {'penalty': 'l2', 'C': 1}, {'penalty': 'l1', 'C': 10}]


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    9.2s finished
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [7]:
# Heart Logistic Regression Tuning
best_param_grid_heart = hyper_tune(heart_X_train, heart_y_train, LogisticRegression(), log_param_grid)

with open('params/log/best_param_grid_heart', 'w') as f:
    json.dump(best_param_grid_heart, f)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 10 is smaller than n_iter=20. Running 10 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1925s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0855s.) Setting batch_size=4.


Best 3 params: [{'penalty': 'l1', 'C': 1}, {'penalty': 'l2', 'C': 1}, {'penalty': 'l1', 'C': 10}]


[Parallel(n_jobs=-1)]: Done  39 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0561s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done  65 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done  85 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.9s finished
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [8]:
# Adult Logistic Regression Tuning
best_param_grid_adult = hyper_tune(adult_X_train, adult_y_train, LogisticRegression(), log_param_grid)

with open('params/log/best_param_grid_adult', 'w') as f:
    json.dump(best_param_grid_adult, f)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 10 is smaller than n_iter=20. Running 10 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    9.7s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   14.6s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   25.5s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   29.7s
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:   34.5s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:   38.4s
[Parallel(n_jobs=-1)]: Done 100 

Best 3 params: [{'penalty': 'l1', 'C': 1}, {'penalty': 'l2', 'C': 1}, {'penalty': 'l1', 'C': 10}]


### Decision Tree

In [9]:
tree_param_grid = {
    "criterion" : ['gini', 'entropy'],
    "max_depth" : [4,6,8,12],
}

In [10]:
# Mushroom Decision Tree Tuning
best_param_grid_mush = hyper_tune(mush_X_train, mush_y_train, DecisionTreeClassifier(), tree_param_grid)

with open('params/tree/best_param_grid_mush', 'w') as f:
    json.dump(best_param_grid_mush, f)

Fitting 10 folds for each of 8 candidates, totalling 80 fits


/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 8 is smaller than n_iter=20. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1993s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    2.7s


Best 3 params: [{'max_depth': 4, 'criterion': 'gini'}, {'max_depth': 6, 'criterion': 'gini'}, {'max_depth': 8, 'criterion': 'gini'}]


[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    3.1s finished


In [11]:
# Heart Decision Tree Tuning
best_param_grid_heart = hyper_tune(heart_X_train, heart_y_train, DecisionTreeClassifier(), tree_param_grid)

with open('params/tree/best_param_grid_heart', 'w') as f:
    json.dump(best_param_grid_heart, f)

Fitting 10 folds for each of 8 candidates, totalling 80 fits


/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 8 is smaller than n_iter=20. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1751s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0386s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:    1.6s


Best 3 params: [{'max_depth': 4, 'criterion': 'gini'}, {'max_depth': 6, 'criterion': 'gini'}, {'max_depth': 8, 'criterion': 'gini'}]


[Parallel(n_jobs=-1)]: Batch computation too fast (0.0480s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    1.7s finished
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [12]:
# Adult Decision Tree Tuning
best_param_grid_adult = hyper_tune(adult_X_train, adult_y_train, DecisionTreeClassifier(), tree_param_grid)

with open('params/tree/best_param_grid_adult', 'w') as f:
    json.dump(best_param_grid_adult, f)

Fitting 10 folds for each of 8 candidates, totalling 80 fits


/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 8 is smaller than n_iter=20. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    6.9s


Best 3 params: [{'max_depth': 4, 'criterion': 'gini'}, {'max_depth': 6, 'criterion': 'gini'}, {'max_depth': 8, 'criterion': 'gini'}]


[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    8.0s finished


### Perceptron

In [13]:
perc_param_grid = {
    "penalty" : [None, "l1", "l2", "elasticnet"],
    "alpha" : [0.001, 0.0001, 0.00001],
    "max_iter" : [500, 1000, 2000],
    "tol" : [1e-4, 1e-3, 1e-2],
    "early_stopping" : [True, False]
}

In [14]:
# Mushroom Perceptron Tuning
best_param_grid_mush = hyper_tune(mush_X_train, mush_y_train, Perceptron(), perc_param_grid)

with open('params/perc/best_param_grid_mush', 'w') as f:
    json.dump(best_param_grid_mush, f)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1983s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1249s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 108 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:    3.9s


Best 3 params: [{'tol': 0.0001, 'penalty': 'l1', 'max_iter': 1000, 'early_stopping': False, 'alpha': 1e-05}, {'tol': 0.0001, 'penalty': None, 'max_iter': 1000, 'early_stopping': True, 'alpha': 1e-05}, {'tol': 0.001, 'penalty': 'l1', 'max_iter': 500, 'early_stopping': True, 'alpha': 1e-05}]


[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    4.6s finished


In [15]:
# Heart Perceptron Tuning
best_param_grid_heart = hyper_tune(heart_X_train, heart_y_train, Perceptron(), perc_param_grid)

with open('params/perc/best_param_grid_heart', 'w') as f:
    json.dump(best_param_grid_heart, f)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1984s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0357s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  44 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0867s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0442s.) Setting batch_size=16.
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    1.6s


Best 3 params: [{'tol': 0.01, 'penalty': 'l1', 'max_iter': 1000, 'early_stopping': True, 'alpha': 0.001}, {'tol': 0.01, 'penalty': 'elasticnet', 'max_iter': 2000, 'early_stopping': False, 'alpha': 1e-05}, {'tol': 0.001, 'penalty': None, 'max_iter': 2000, 'early_stopping': True, 'alpha': 0.0001}]


[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    1.9s finished
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [16]:
# Adult Perceptron Tuning
best_param_grid_adult = hyper_tune(adult_X_train, adult_y_train, Perceptron(), perc_param_grid)

with open('params/perc/best_param_grid_adult', 'w') as f:
    json.dump(best_param_grid_adult, f)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:   13.5s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   14.7s
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:   16.9s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   18.0s
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:   

Best 3 params: [{'tol': 0.001, 'penalty': 'l2', 'max_iter': 500, 'early_stopping': True, 'alpha': 1e-05}, {'tol': 0.001, 'penalty': 'l1', 'max_iter': 500, 'early_stopping': False, 'alpha': 1e-05}, {'tol': 0.0001, 'penalty': None, 'max_iter': 500, 'early_stopping': True, 'alpha': 0.0001}]


[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:   21.7s finished


### Multi-Layer Perceptron

In [17]:
mlp_param_grid = {
    "hidden_layer_sizes" : [(100,), (50,), (200,), (25,)],
    "activation" : ["identity", "logistic", "tanh", "relu"],
    "solver" : ["lbfgs", "sgd", "adam"],
    "max_iter" : [200, 100, 300],
    "tol" : [1e-4, 1e-3, 1e-5],
    "early_stopping" : [True, False]
}

In [18]:
# Mushroom Perceptron Tuning
best_param_grid_mush = hyper_tune(mush_X_train, mush_y_train, MLPClassifier(), mlp_param_grid)

with open('params/mlp/best_param_grid_mush', 'w') as f:
    json.dump(best_param_grid_mush, f)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   14.7s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   23.5s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   28.5s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   49.0s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:  2

Best 3 params: [{'tol': 1e-05, 'solver': 'lbfgs', 'max_iter': 300, 'hidden_layer_sizes': (200,), 'early_stopping': False, 'activation': 'identity'}, {'tol': 0.001, 'solver': 'adam', 'max_iter': 200, 'hidden_layer_sizes': (50,), 'early_stopping': False, 'activation': 'logistic'}, {'tol': 0.0001, 'solver': 'sgd', 'max_iter': 200, 'hidden_layer_sizes': (50,), 'early_stopping': False, 'activation': 'logistic'}]


In [19]:
# Heart Perceptron Tuning
best_param_grid_heart = hyper_tune(heart_X_train, heart_y_train, MLPClassifier(), mlp_param_grid)

with open('params/mlp/best_param_grid_heart', 'w') as f:
    json.dump(best_param_grid_heart, f)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1781s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  54 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 102 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 128 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done 188 tasks      | elapsed:    9.7s
[Parallel(n_jobs=-1)]: Done 193 out of 200 | elapsed:    9.9s remaining:    0.4s
[Parallel(n_jobs

Best 3 params: [{'tol': 1e-05, 'solver': 'adam', 'max_iter': 200, 'hidden_layer_sizes': (25,), 'early_stopping': False, 'activation': 'logistic'}, {'tol': 1e-05, 'solver': 'sgd', 'max_iter': 200, 'hidden_layer_sizes': (50,), 'early_stopping': False, 'activation': 'tanh'}, {'tol': 0.0001, 'solver': 'adam', 'max_iter': 200, 'hidden_layer_sizes': (50,), 'early_stopping': False, 'activation': 'tanh'}]


In [20]:
# Adult Perceptron Tuning
best_param_grid_adult = hyper_tune(adult_X_train, adult_y_train, MLPClassifier(), mlp_param_grid)

with open('params/mlp/best_param_grid_adult', 'w') as f:
    json.dump(best_param_grid_adult, f)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed: 10.4min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 10.7min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed: 11

Best 3 params: [{'tol': 1e-05, 'solver': 'sgd', 'max_iter': 200, 'hidden_layer_sizes': (50,), 'early_stopping': False, 'activation': 'tanh'}, {'tol': 0.0001, 'solver': 'sgd', 'max_iter': 100, 'hidden_layer_sizes': (200,), 'early_stopping': True, 'activation': 'identity'}, {'tol': 0.001, 'solver': 'adam', 'max_iter': 300, 'hidden_layer_sizes': (200,), 'early_stopping': True, 'activation': 'identity'}]


### KNN

In [21]:
knn_param_grid = {
    "n_neighbors" : [1, 3, 5, 9, 15, 25, 50, 75, 100],
}

In [22]:
# Mushroom Decision Tree Tuning
best_param_grid_mush = hyper_tune(mush_X_train, mush_y_train, KNeighborsClassifier(), knn_param_grid)

with open('params/knn/best_param_grid_mush', 'w') as f:
    json.dump(best_param_grid_mush, f)

Fitting 10 folds for each of 9 candidates, totalling 90 fits


/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 9 is smaller than n_iter=20. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   12.5s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   15.8s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   20.7s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   23.8s
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:   28.4s


Best 3 params: [{'n_neighbors': 1}, {'n_neighbors': 3}, {'n_neighbors': 5}]


[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:   31.9s finished


In [23]:
# Heart Decision Tree Tuning
best_param_grid_heart = hyper_tune(heart_X_train, heart_y_train, KNeighborsClassifier(), knn_param_grid)

with open('params/knn/best_param_grid_heart', 'w') as f:
    json.dump(best_param_grid_heart, f)

Fitting 10 folds for each of 9 candidates, totalling 90 fits


/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 9 is smaller than n_iter=20. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1963s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0193s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  44 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0828s.) Setting batch_size=8.


Best 3 params: [{'n_neighbors': 1}, {'n_neighbors': 3}, {'n_neighbors': 5}]


[Parallel(n_jobs=-1)]: Done  71 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    1.4s finished
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [24]:
# Adult Decision Tree Tuning
best_param_grid_adult = hyper_tune(adult_X_train, adult_y_train, KNeighborsClassifier(), knn_param_grid)

with open('params/knn/best_param_grid_adult', 'w') as f:
    json.dump(best_param_grid_adult, f)

Fitting 10 folds for each of 9 candidates, totalling 90 fits


/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 9 is smaller than n_iter=20. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   34.6s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   55.3s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  6.9min finished


Best 3 params: [{'n_neighbors': 1}, {'n_neighbors': 3}, {'n_neighbors': 5}]


### Random Forest

In [25]:
rf_param_grid = {
    'bootstrap': [True, False],
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
}

In [26]:
# Mushroom Decision Tree Tuning
best_param_grid_mush = hyper_tune(mush_X_train, mush_y_train, RandomForestClassifier(), rf_param_grid)

with open('params/rf/best_param_grid_mush', 'w') as f:
    json.dump(best_param_grid_mush, f)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   14.2s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   18.5s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   26.7s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   37.8s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   58.6s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:  5

Best 3 params: [{'n_estimators': 1200, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 30, 'bootstrap': True}, {'n_estimators': 600, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False}, {'n_estimators': 1600, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 90, 'bootstrap': True}]


In [27]:
# Heart Decision Tree Tuning
best_param_grid_heart = hyper_tune(heart_X_train, heart_y_train, RandomForestClassifier(), rf_param_grid)

with open('params/rf/best_param_grid_heart', 'w') as f:
    json.dump(best_param_grid_heart, f)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   15.9s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   21.7s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   28.9s
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:   31.9s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:   33.8s
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:   42.8s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   52.1s
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:  1

Best 3 params: [{'n_estimators': 600, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False}, {'n_estimators': 1400, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': True}, {'n_estimators': 1400, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 70, 'bootstrap': False}]


In [28]:
# Adult Decision Tree Tuning
best_param_grid_adult = hyper_tune(adult_X_train, adult_y_train, RandomForestClassifier(), rf_param_grid)

with open('params/rf/best_param_grid_adult', 'w') as f:
    json.dump(best_param_grid_adult, f)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  9.4min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 11.9min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 16.7min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 21.9min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 23.3min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 24.7min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 29.5min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed: 34.8min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 40.0min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed: 44

Best 3 params: [{'n_estimators': 1000, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 10, 'bootstrap': True}, {'n_estimators': 1600, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 90, 'bootstrap': True}, {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False}]


## Evaluate with best Params

### SVM

In [ ]:
with open('params/svm/best_param_grid_mush', 'r') as f:
    best_param_grid_mush = json.load(f)
    
with open('params/svm/best_param_grid_heart', 'r') as f:
    best_param_grid_heart = json.load(f)
    
with open('params/svm/best_param_grid_adult', 'r') as f:
    best_param_grid_adult = json.load(f)

In [ ]:
# Mushroom SVM
for i in range(len(best_param_grid_mush)):
    print("Best params:", best_param_grid_mush[i], "\n")
    classifier = clf_SVM(best_param_grid_mush[i])
    train_model(classifier, mush_X_train, mush_y_train)
    acc, prec, rec, f = evalModel(classifier, mush_X_test, mush_y_test)
    print(f"Accuracy: {acc} \nPrecision: {prec} \nRecall: {rec} \nF1 Score {f}")

In [ ]:
# Heart SVM
for i in range(len(best_param_grid_heart)):
    print("Best params:", best_param_grid_heart[i], "\n")
    classifier = clf_SVM(best_param_grid_heart[i])
    train_model(classifier, heart_X_train, heart_y_train)
    acc, prec, rec, f = evalModel(classifier, heart_X_test, heart_y_test)
    print(f"Accuracy: {acc} \nPrecision: {prec} \nRecall: {rec} \nF1 Score {f}")

In [ ]:
# Adult SVM
for i in range(len(best_param_grid_adult)):
    print("Best params:", best_param_grid_adult[i], "\n")
    classifier = clf_SVM(best_param_grid_adult[i])
    train_model(classifier, adult_X_train, adult_y_train)
    acc, prec, rec, f = evalModel(classifier, adult_X_test, adult_y_test)
    print(f"Accuracy: {acc} \nPrecision: {prec} \nRecall: {rec} \nF1 Score {f}")

### Logistic Regression

In [ ]:
with open('params/log/best_param_grid_mush', 'r') as f:
    best_param_grid_mush = json.load(f)
    
with open('params/log/best_param_grid_heart', 'r') as f:
    best_param_grid_heart = json.load(f)
    
with open('params/log/best_param_grid_adult', 'r') as f:
    best_param_grid_adult = json.load(f)

In [ ]:
# Mushroom Logistic Regression Training/Eval
for i in range(len(best_param_grid_mush)):
    print("Best params:", best_param_grid_mush[i], "\n")
    classifier = clf_log(best_param_grid_mush[i])
    train_model(classifier, mush_X_train, mush_y_train)
    acc, prec, rec, f = evalModel(classifier, mush_X_test, mush_y_test)
    print(f"Accuracy: {acc} \nPrecision: {prec} \nRecall: {rec} \nF1 Score {f}")

In [ ]:
# Heart Logistic Regression Training/Eval
for i in range(len(best_param_grid_heart)):
    print("Best params:", best_param_grid_heart[i], "\n")
    classifier = clf_log(best_param_grid_heart[i])
    train_model(classifier, heart_X_train, heart_y_train)
    acc, prec, rec, f = evalModel(classifier, heart_X_test, heart_y_test)
    print(f"Accuracy: {acc} \nPrecision: {prec} \nRecall: {rec} \nF1 Score {f}")

In [ ]:
# Adult Logistic Regression Training/Eval
for i in range(len(best_param_grid_adult)):
    print("Best params:", best_param_grid_adult[i], "\n")
    classifier = clf_log(best_param_grid_adult[i])
    train_model(classifier, adult_X_train, adult_y_train)
    acc, prec, rec, f = evalModel(classifier, adult_X_test, adult_y_test)
    print(f"Accuracy: {acc} \nPrecision: {prec} \nRecall: {rec} \nF1 Score {f}")

### Decision Tree

In [ ]:
with open('params/tree/best_param_grid_mush', 'r') as f:
    best_param_grid_mush = json.load(f)
    
with open('params/tree/best_param_grid_heart', 'r') as f:
    best_param_grid_heart = json.load(f)
    
with open('params/tree/best_param_grid_adult', 'r') as f:
    best_param_grid_adult = json.load(f)

In [ ]:
# Mushroom Decision Tree Training/Eval
for i in range(len(best_param_grid_mush)):
    print("Best params:", best_param_grid_mush[i], "\n")
    classifier = clf_tree(best_param_grid_mush[i])
    train_model(classifier, mush_X_train, mush_y_train)
    acc, prec, rec, f = evalModel(classifier, mush_X_test, mush_y_test)
    print(f"Accuracy: {acc} \nPrecision: {prec} \nRecall: {rec} \nF1 Score {f}")

In [ ]:
# Heart Decision Tree Training/Eval
for i in range(len(best_param_grid_heart)):
    print("Best params:", best_param_grid_heart[i], "\n")
    classifier = clf_tree(best_param_grid_heart[i])
    train_model(classifier, heart_X_train, heart_y_train)
    acc, prec, rec, f = evalModel(classifier, heart_X_test, heart_y_test)
    print(f"Accuracy: {acc} \nPrecision: {prec} \nRecall: {rec} \nF1 Score {f}")

In [ ]:
# Adult Decision Tree Training/Eval
for i in range(len(best_param_grid_adult)):
    print("Best params:", best_param_grid_adult[i], "\n")
    classifier = clf_tree(best_param_grid_adult[i])
    train_model(classifier, adult_X_train, adult_y_train)
    acc, prec, rec, f = evalModel(classifier, adult_X_test, adult_y_test)
    print(f"Accuracy: {acc} \nPrecision: {prec} \nRecall: {rec} \nF1 Score {f}")

### Perceptron

In [ ]:
with open('params/perc/best_param_grid_mush', 'r') as f:
    best_param_grid_mush = json.load(f)
    
with open('params/perc/best_param_grid_heart', 'r') as f:
    best_param_grid_heart = json.load(f)
    
with open('params/perc/best_param_grid_adult', 'r') as f:
    best_param_grid_adult = json.load(f)

In [ ]:
# Mushroom Decision Tree Training/Eval
for i in range(len(best_param_grid_mush)):
    print("Best params:", best_param_grid_mush[i], "\n")
    classifier = clf_perc(best_param_grid_mush[i])
    train_model(classifier, mush_X_train, mush_y_train)
    acc, prec, rec, f = evalModel(classifier, mush_X_test, mush_y_test)
    print(f"Accuracy: {acc} \nPrecision: {prec} \nRecall: {rec} \nF1 Score {f}")

In [ ]:
# Heart Decision Tree Training/Eval
for i in range(len(best_param_grid_heart)):
    print("Best params:", best_param_grid_heart[i], "\n")
    classifier = clf_perc(best_param_grid_heart[i])
    train_model(classifier, heart_X_train, heart_y_train)
    acc, prec, rec, f = evalModel(classifier, heart_X_test, heart_y_test)
    print(f"Accuracy: {acc} \nPrecision: {prec} \nRecall: {rec} \nF1 Score {f}")

In [ ]:
# Adult Perceptron Training/Eval
for i in range(len(best_param_grid_adult)):
    print("Best params:", best_param_grid_adult[i], "\n")
    classifier = clf_perc(best_param_grid_adult[i])
    train_model(classifier, adult_X_train, adult_y_train)
    acc, prec, rec, f = evalModel(classifier, adult_X_test, adult_y_test)
    print(f"Accuracy: {acc} \nPrecision: {prec} \nRecall: {rec} \nF1 Score {f}")

### Multi-Layer Perceptron

In [ ]:
with open('params/mlp/best_param_grid_mush', 'r') as f:
    best_param_grid_mush = json.load(f)
    
with open('params/mlp/best_param_grid_heart', 'r') as f:
    best_param_grid_heart = json.load(f)
    
with open('params/mlp/best_param_grid_adult', 'r') as f:
    best_param_grid_adult = json.load(f)

In [ ]:
# Mushroom MLP Training/Eval
for i in range(len(best_param_grid_mush)):
    print("Best params:", best_param_grid_mush[i], "\n")
    classifier = clf_mlp(best_param_grid_mush[i])
    train_model(classifier, mush_X_train, mush_y_train)
    acc, prec, rec, f = evalModel(classifier, mush_X_test, mush_y_test)
    print(f"Accuracy: {acc} \nPrecision: {prec} \nRecall: {rec} \nF1 Score {f}")

In [ ]:
# Heart MLP Training/Eval
for i in range(len(best_param_grid_heart)):
    print("Best params:", best_param_grid_heart[i], "\n")
    classifier = clf_mlp(best_param_grid_heart[i])
    train_model(classifier, heart_X_train, heart_y_train)
    acc, prec, rec, f = evalModel(classifier, heart_X_test, heart_y_test)
    print(f"Accuracy: {acc} \nPrecision: {prec} \nRecall: {rec} \nF1 Score {f}")

In [ ]:
# Adult MLP Training/Eval
for i in range(len(best_param_grid_adult)):
    print("Best params:", best_param_grid_adult[i], "\n")
    classifier = clf_mlp(best_param_grid_adult[i])
    train_model(classifier, adult_X_train, adult_y_train)
    acc, prec, rec, f = evalModel(classifier, adult_X_test, adult_y_test)
    print(f"Accuracy: {acc} \nPrecision: {prec} \nRecall: {rec} \nF1 Score {f}")

### KNN

In [ ]:
with open('params/knn/best_param_grid_mush', 'r') as f:
    best_param_grid_mush = json.load(f)
    
with open('params/knn/best_param_grid_heart', 'r') as f:
    best_param_grid_heart = json.load(f)
    
with open('params/knn/best_param_grid_adult', 'r') as f:
    best_param_grid_adult = json.load(f)

In [ ]:
# Mushroom KNN Training/Eval
for i in range(len(best_param_grid_mush)):
    print("Best params:", best_param_grid_mush[i], "\n")
    classifier = clf_knn(best_param_grid_mush[i])
    train_model(classifier, mush_X_train, mush_y_train)
    acc, prec, rec, f = evalModel(classifier, mush_X_test, mush_y_test)
    print(f"Accuracy: {acc} \nPrecision: {prec} \nRecall: {rec} \nF1 Score {f}")

In [ ]:
# Heart KNN Training/Eval
for i in range(len(best_param_grid_heart)):
    print("Best params:", best_param_grid_heart[i], "\n")
    classifier = clf_knn(best_param_grid_heart[i])
    train_model(classifier, heart_X_train, heart_y_train)
    acc, prec, rec, f = evalModel(classifier, heart_X_test, heart_y_test)
    print(f"Accuracy: {acc} \nPrecision: {prec} \nRecall: {rec} \nF1 Score {f}")

In [ ]:
# Adult KNN Training/Eval
for i in range(len(best_param_grid_adult)):
    print("Best params:", best_param_grid_adult[i], "\n")
    classifier = clf_knn(best_param_grid_adult[i])
    train_model(classifier, adult_X_train, adult_y_train)
    acc, prec, rec, f = evalModel(classifier, adult_X_test, adult_y_test)
    print(f"Accuracy: {acc} \nPrecision: {prec} \nRecall: {rec} \nF1 Score {f}")

### Random Forest

In [ ]:
with open('params/rf/best_param_grid_mush', 'r') as f:
    best_param_grid_mush = json.load(f)
    
with open('params/rf/best_param_grid_heart', 'r') as f:
    best_param_grid_heart = json.load(f)
    
with open('params/rf/best_param_grid_adult', 'r') as f:
    best_param_grid_adult = json.load(f)

In [ ]:
# Mushroom RF Training/Eval
for i in range(len(best_param_grid_mush)):
    print("Best params:", best_param_grid_mush[i], "\n")
    classifier = clf_rf(best_param_grid_mush[i])
    train_model(classifier, mush_X_train, mush_y_train)
    acc, prec, rec, f = evalModel(classifier, mush_X_test, mush_y_test)
    print(f"Accuracy: {acc} \nPrecision: {prec} \nRecall: {rec} \nF1 Score {f}")

In [ ]:
# Heart RF Training/Eval
for i in range(len(best_param_grid_heart)):
    print("Best params:", best_param_grid_heart[i], "\n")
    classifier = clf_rf(best_param_grid_heart[i])
    train_model(classifier, heart_X_train, heart_y_train)
    acc, prec, rec, f = evalModel(classifier, heart_X_test, heart_y_test)
    print(f"Accuracy: {acc} \nPrecision: {prec} \nRecall: {rec} \nF1 Score {f}")

In [ ]:
# Adult RF Training/Eval
for i in range(len(best_param_grid_adult)):
    print("Best params:", best_param_grid_adult[i], "\n")
    classifier = clf_rf(best_param_grid_adult[i])
    train_model(classifier, adult_X_train, adult_y_train)
    acc, prec, rec, f = evalModel(classifier, adult_X_test, adult_y_test)
    print(f"Accuracy: {acc} \nPrecision: {prec} \nRecall: {rec} \nF1 Score {f}")

## Evaluation Pipeline

- 3 trials
    - 3 datasets
        - 4 models
            - 3 splits (80/20, 20/80, 50/50)    

In [29]:
num_trials = 3
datasets = ['adult', 'mush', 'heart']
splits = [0.2, 0.8]
models = ['log', 'svm', 'tree', 'perc', 'mlp', 'knn', 'rf']

In [30]:
"""
For i in three different datasets
   For j in three different partitions (20/80,50/50,80/20):
        For t in three different trials
            For c in three different classifiers
                 cross validate
                 find the optimal hyper-parameter
                 train using the hyper-parameter above
                 obtain the training and validation accuracy/error
                 test
                 obtain the testing accuracy
       compute the averaged accuracy (training, validation, and testing) for each classifier c out of three trials
       rank order the classifiers
""";

In [31]:
# Loop through datasets
dataset_scores = {}
for dataset in datasets:
    
    # Loop through dataset splits
    split_scores = {}
    for split in splits:

        # Prepare data splits
        X, y, X_train, X_test, y_train, y_test = pre_process(dataset=dataset, split=split)
        
        # Loop through trials
        trial_scores = {}
        for i in range(num_trials):
            
            # Loop through models
            model_scores = {}
            for model in models:
                
                # Load best model params for given model and dataset
                with open(f'params/{model}/best_param_grid_{dataset}', 'r') as f:
                    best_param_grid = json.load(f)
    
                    # Create classifier
                    classifier = clf(model=model, param_grid=best_param_grid[i])

                    # Train classifier
                    print(f"Training {dataset}-{split}-{i}-{model}")
                    train_model(classifier, X_train, y_train)

                    # Evaluate classifier
                    print(f"Evaluating {dataset}-{split}-{i}-{model}")
                    acc, prec, rec, f = evalModel(classifier, X_test, y_test)
                    classifier_eval = {"accuracy" : acc, "precision": prec, "recall" : rec, "f1_score" : f}              
                    
                # Add evaluation scores for given model
                model_scores.update({f"model_{model}" : classifier_eval})
                
            # Add model scores for given trial
            trial_scores.update({f"trial_{i}" : model_scores})

        # Add trial scores for given model
        split_scores.update({f"split_{split}": trial_scores})

    # Add split scores for given dataset
    dataset_scores.update({f"data_{dataset}": split_scores})
    
with open('scores/dataset_scores', 'w') as f:
    json.dump(dataset_scores, f)

Training adult-0.2-0-log
Evaluating adult-0.2-0-log
Training adult-0.2-0-svm


/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:241: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Evaluating adult-0.2-0-svm
Training adult-0.2-0-tree
Evaluating adult-0.2-0-tree
Training adult-0.2-0-perc
Evaluating adult-0.2-0-perc
Training adult-0.2-0-mlp


/usr/local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Evaluating adult-0.2-0-mlp
Training adult-0.2-0-knn
Evaluating adult-0.2-0-knn
Training adult-0.2-0-rf
Evaluating adult-0.2-0-rf
Training adult-0.2-1-log
Evaluating adult-0.2-1-log
Training adult-0.2-1-svm
Evaluating adult-0.2-1-svm
Training adult-0.2-1-tree
Evaluating adult-0.2-1-tree
Training adult-0.2-1-perc
Evaluating adult-0.2-1-perc
Training adult-0.2-1-mlp
Evaluating adult-0.2-1-mlp
Training adult-0.2-1-knn
Evaluating adult-0.2-1-knn
Training adult-0.2-1-rf
Evaluating adult-0.2-1-rf
Training adult-0.2-2-log
Evaluating adult-0.2-2-log
Training adult-0.2-2-svm
Evaluating adult-0.2-2-svm
Training adult-0.2-2-tree
Evaluating adult-0.2-2-tree
Training adult-0.2-2-perc
Evaluating adult-0.2-2-perc
Training adult-0.2-2-mlp
Evaluating adult-0.2-2-mlp
Training adult-0.2-2-knn
Evaluating adult-0.2-2-knn
Training adult-0.2-2-rf
Evaluating adult-0.2-2-rf
Training adult-0.8-0-log
Evaluating adult-0.8-0-log
Training adult-0.8-0-svm


...

Evaluating heart-0.8-0-mlp
Training heart-0.8-0-knn
Evaluating heart-0.8-0-knn
Training heart-0.8-0-rf
Evaluating heart-0.8-0-rf
Training heart-0.8-1-log
Evaluating heart-0.8-1-log
Training heart-0.8-1-svm
Evaluating heart-0.8-1-svm
Training heart-0.8-1-tree
Evaluating heart-0.8-1-tree
Training heart-0.8-1-perc
Evaluating heart-0.8-1-perc
Training heart-0.8-1-mlp


/usr/local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Evaluating heart-0.8-1-mlp
Training heart-0.8-1-knn
Evaluating heart-0.8-1-knn
Training heart-0.8-1-rf
Evaluating heart-0.8-1-rf
Training heart-0.8-2-log
Evaluating heart-0.8-2-log
Training heart-0.8-2-svm
Evaluating heart-0.8-2-svm
Training heart-0.8-2-tree
Evaluating heart-0.8-2-tree
Training heart-0.8-2-perc
Evaluating heart-0.8-2-perc
Training heart-0.8-2-mlp


/usr/local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Evaluating heart-0.8-2-mlp
Training heart-0.8-2-knn
Evaluating heart-0.8-2-knn
Training heart-0.8-2-rf
Evaluating heart-0.8-2-rf


In [32]:
with open('scores/dataset_scores', 'r') as f:
    dataset_scores = json.load(f)

In [33]:
dataset_scores

{'data_adult': {'split_0.2': {'trial_0': {'model_log': {'accuracy': 1.0,
     'precision': 1.0,
     'recall': 1.0,
     'f1_score': 1.0},
    'model_svm': {'accuracy': 1.0,
     'precision': 1.0,
     'recall': 1.0,
     'f1_score': 1.0},
    'model_tree': {'accuracy': 1.0,
     'precision': 1.0,
     'recall': 1.0,
     'f1_score': 1.0},
    'model_perc': {'accuracy': 0.9986692599037773,
     'precision': 0.9992717902755994,
     'recall': 0.9924065420560748,
     'f1_score': 0.9958098499293646},
    'model_mlp': {'accuracy': 0.999181083017709,
     'precision': 0.999551619773568,
     'recall': 0.9953271028037383,
     'f1_score': 0.9974282914789022},
    'model_knn': {'accuracy': 0.954550107482854,
     'precision': 0.8609517384899257,
     'recall': 0.8510091974234841,
     'f1_score': 0.8558918206877745},
    'model_rf': {'accuracy': 0.9353055583990173,
     'precision': 0.9668936616029336,
     'recall': 0.6308411214953271,
     'f1_score': 0.690287459417376}},
   'trial_1': {'m

## Dataset Results

In [ ]:
def query_data(_dataset_name, _model_name, dataset_scores=dataset_scores):
    split_02 = []
    split_05 = []
    split_08 = []
    for split_name, split_data in dataset_scores["data_adult"].items():
        for trial_name, trial_data in split_data.items():
            if split_name=="split_0.2":
                split_02.append(trial_data[_model_name]["accuracy"])
            elif split_name=="split_0.5":
                split_05.append(trial_data[_model_name]["accuracy"])
            elif split_name=="split_0.8":
                split_08.append(trial_data[_model_name]["accuracy"])

    df = pd.DataFrame([split_02, split_05, split_08], columns=["Trial 1", "Trial 2", "Trial 3"])
    df["Trial_Avg"] = df.T.mean()
    df.loc[3] = df.mean()
    df = df.rename({0:"Split 0.2", 1:"Split 0.5", 2: "Split 0.8", 3: "Split_Avg"})
    return df

In [34]:
def data_table(_split_name, dataset_scores=dataset_scores, models=models):
    dataset_adult = []
    dataset_heart = []
    dataset_mush = []
    for dataset_name, dataset_data in dataset_scores.items():
        for trial_name, trial_data in dataset_data[_split_name].items():
            for model_name, model_data in trial_data.items():
                print(dataset_name, trial_name, model_name, model_data)
            if dataset_name == "data_adult":
                pass
#                 print(trial_data)
#                 dataset_adult.append()
#             print(f"{dataset_name}-{_split_name}-{trial_name}")

    return
    df = pd.DataFrame([split_02, split_05, split_08], columns=["Trial 1", "Trial 2", "Trial 3"])
    df["Trial_Avg"] = df.T.mean()
    df.loc[3] = df.mean()
    df = df.rename({0:"Split 0.2", 1:"Split 0.5", 2: "Split 0.8", 3: "Split_Avg"})
    return df

In [ ]:
dataset_scores

In [35]:
data_table("split_0.2")

data_adult trial_0 model_log {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0}
data_adult trial_0 model_svm {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0}
data_adult trial_0 model_tree {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0}
data_adult trial_0 model_perc {'accuracy': 0.9986692599037773, 'precision': 0.9992717902755994, 'recall': 0.9924065420560748, 'f1_score': 0.9958098499293646}
data_adult trial_0 model_mlp {'accuracy': 0.999181083017709, 'precision': 0.999551619773568, 'recall': 0.9953271028037383, 'f1_score': 0.9974282914789022}
data_adult trial_0 model_knn {'accuracy': 0.954550107482854, 'precision': 0.8609517384899257, 'recall': 0.8510091974234841, 'f1_score': 0.8558918206877745}
data_adult trial_0 model_rf {'accuracy': 0.9353055583990173, 'precision': 0.9668936616029336, 'recall': 0.6308411214953271, 'f1_score': 0.690287459417376}
data_adult trial_1 model_log {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_

### Happiness Survey

In [ ]:
# happy_df = pd.concat({"Log Regression": query_data("data_happy", "model_log"),
#                       "SVM" : query_data("data_happy", "model_svm"),
#                       "Decision Tree" : query_data("data_happy", "model_tree"),
#                       "Perceptron" : query_data("data_happy", "model_perc"),
#                       "MLP" : query_data("data_happy", "model_mlp"),
#                      "KNN" : query_data("data_happy", "model_knn"),
#                      "Random Forest" : query_data("data_happy", "model_rf")})
# happy_df

### Adult

In [ ]:
adult_df = pd.concat({"Log Regression": query_data("data_adult", "model_log"),
                      "SVM" : query_data("data_adult", "model_svm"),
                      "Decision Tree" : query_data("data_adult", "model_tree"),
                      "Perceptron" : query_data("data_adult", "model_perc"),
                      "MLP" : query_data("data_adult", "model_mlp"),
                     "KNN" : query_data("data_adult", "model_knn"),
                     "Random Forest" : query_data("data_adult", "model_rf")})
adult_df

### Mushroom

In [ ]:
mush_df = pd.concat({"Log Regression": query_data("data_mush", "model_log"),
                              "SVM" : query_data("data_mush", "model_svm"),
                    "Decision Tree" : query_data("data_mush", "model_tree"),
                    "Perceptron" : query_data("data_mush", "model_perc"),
                    "MLP" : query_data("data_mush", "model_mlp"),
                    "KNN" : query_data("data_mush", "model_knn"),
                    "Random Forest" : query_data("data_mush", "model_rf")})
mush_df

In [ ]:
heart_df = pd.concat({"Log Regression": query_data("data_heart", "model_log"),
                              "SVM" : query_data("data_heart", "model_svm"),
                     "Decision Tree" : query_data("data_heart", "model_tree"),
                     "Perceptron" : query_data("data_heart", "model_perc"),
                     "MLP" : query_data("data_heart", "model_mlp"),
                     "KNN" : query_data("data_heart", "model_knn"),
                     "Random Forest" : query_data("data_heart", "model_rf")})
heart_df

In [ ]:
main_df = pd.concat({"Adult" : adult_df, "Mush" : mush_df, "Heart" : heart_df})
main_df.to_csv("data/main_df.csv")
main_df

## Instructions

Single person project and no team work.

Report format:

Write a report with >1,000 words (excluding references) including main sections: a) abstract, b) introduction, c) method, d) experiment, e) conclusion, and f) references. You can follow the paper format as e.g leading machine learning journals such as Journal of Machine Learning Research (http://www.jmlr.org/) or IEEE Trans. on Pattern Analysis and Machine Intelligence (http://www.computer.org/web/tpami), or leading conferences like NeurIPS (https://papers.nips.cc/) and ICML (https://icml.cc/). There is no page limit for your report.

Bonus points: 

If you feel that your work deserves bonus points due to reasons such as: a) novel ideas and applications, b) large efforts in your own data collection/preparation, c) state-of-the-art classification results, or d) new algorithms, please create a "Bonus Points" section to specifically describe why you deserve bonus points.

In this project you will choose any three classifiers out of those tested in


We have been discussing the classification problem in the form of two-class classifiers throughout the class. Some classifiers like decision tree, KNN, random forests stay agnostic w.r.t the number of classes but others like SVM and Boosting where explicit objective functions are involved don't.



The basic requirement for the final project is based on the two-class classification problem. If you have additional bandwidth, you can experiment on the multi-class classification setting. When preparing the dataset to train your classifier (two-class), please try to merge the labels into two groups, positives and negatives, if your dataset happens to consist multi-class labels.



Train your classifiers using the setting (not all metrics are needed) described in the empirical study by Caruana and Niculescu-Mizil. You are supposed to reproduce consistent results as in the paper. However, do expect some small variations. When evaluating the algorithms, you don’t need to use all the metrics that were reported in the paper. Using one metric, e.g. the classification accuracy, is sufficient. Please report the cross-validated classification results with the corresponding learned hyper-parameters.

Note that since you are choosing your own libraries for the classifiers, there are implementation details that will affect the classification results. Even the same SVM but with different implementations, you won't be able to see identical results when trained on the same dataset. Therefore, don't expect the identical results as those in the paper, as you are probably using a subset and not all the features. If you see a bit difference in ranking, it should ok but the overall trend should be consistent, e.g. random forest should do well, more training data leads to better results, knn is not necessarily very bad etc.

If you compute accuracy and follow the basic requirement picking 3 classifiers and 3 datasets. You are looking at 3 trials X 3 classifiers X 3 datasets X 3 partitions (20/80, 50/50, 80/20). Each time you always report the best accuracy under the chosen hyper-parameter. Since for the accuracy is averaged among three 3 trials to rank order the classifiers, you will report 3 classifiers X 3 datasets X 3 partitions  (20/80, 50/50, 80/20)  X 3. accuracies (train, validation, test). When trying to debug, always try to see the training accuracy to see if you are able to at least push the training accuracy high (to overfit the data) as a sanity check making sure your implementation is correct. The heatmaps for your hyper-parameters are the details that do not need to be too carefully compared with. The searching for the hyper-parameters is internal and the final conclusion about the classifiers is based on the best hyper-parameter you have obtained for each time.

Since the exact data setting might have changed, the specific parameters and hyper-parameters reported in Caruana and Niculescu-Mizil paper serve as a guideline but you don't need to try all of them. You can try a few standard ones, as long as your classification results are reasonable. If you pick the multi-layer perceptron as one of your classifiers, note that you may need to increase the number of layers to e.g. 3 and create more neurons in each layer to attain good results, for some datasets.

You can alternatively or additionally adopt the datasets and classifiers reported in a follow-up paper, Caruana et al. ICML 2008.
 
You are encouraged to use Python, but using other programming languages and platforms is ok. The candidate classifiers include:
1. Boosting family classifiers
http://www.mathworks.com/matlabcentral/fileexchange/21317-adaboost
or
https://github.com/dmlc/xgboost
2. Support vector machines
http://www.csie.ntu.edu.tw/~cjlin/libsvm/
3. Random Forests
http://www.stat.berkeley.edu/~breiman/RandomForests/
4. Decision Tree
http://www.rulequest.com/Personal/ (please see also see a sample matlab code in the attachment)
5. K-nearest neighbors
http://www.mathworks.com/matlabcentral/fileexchange/19345-efficient-k-nearest-neighbor-searchusing-jit
6. Neural Nets
http://www.cs.colostate.edu/~anderson/code/
http://www.mathworks.com/products/neural-network/code-examples.html
7. Logistic regression classifier
8. Bagging family

The links above are for your reference. You can implement your own classifier or download other
versions you like online (But you need to make sure the code online is reliable). You are supposed to
write a formal report describing about the experiments you run and the corresponding results (plus
code).


Grading
Note that if you do well by satisfying the minimum requirement e.g. 3 classifiers on 3 datasets with cross-validation, you will receive a decent score but not the full 100 points. We are looking for something a bit more and please see the guideline below.

When reporting the experimental results, there are two main sets of comparisons we are looking for:
a. For each dataset on each paritition, show the comparison for different algorithms, and hopefully be consistent with the findings in the paper with Random Forests being the best etc.
b. For each classifier on each partition, show the comparison on different partitions and you are supposed to show the increase of test accuracy (decrease of test error) with more training data and less test data.

Note that the performance and function calls vary due to the particular ML libraries you are using. For example, the same SVM classifier provided in different toolboxes might result in different errors even trained on the same dataset. But the overall differences should be reasonable and interpretable. You may obtain a ranking that is somewhat different from that in the paper, due to differences in detailed implementation of the classifiers, different training sizes, features ect. But the overall trend should be explainable. For example, random forest usually has a pretty good performance; knn might not be as bad as you had thought, kernel-based SVM is sometimes sensitive to the hyper-parameters; using more data in training will lead to improvement, especially on difficult cases.

The merit and grading of your project can be judged from aspects described below that are common
when reviewing a paper:
1. How challenging and large are the datasets you are studying? (10 points)
2. Any aspects that are new in terms of algorithm development, uniqueness of the data, or new
applications? (10 points)
3. Is your experimental design comprehensive? Have you done thoroughly experiments in tuning
hyper-parameters and performing cross validation (you should also try different data partitions, e.g 20% training and 80% testing, 50% training and 50% testing, and 80% training and 20% testing for multiple rounds, e.g. 3 times each for the above three partitions and compute average scores to remove potentials of having accidental results); try to report both the training and testing errors after cross-validation; it is encouraged to also report the training and validation errors during cross-validation using classification error/accuracy curves w.r.t. the hyper-parameters. (50 points)
4. Is your report written in a professional way with sections including abstract, introduction, data
and problem description, method description, experiments, conclusion, and references? (30
points)
5. Bonus points will be assigned to projects in which new ideas have been developed and implemented, or thorough experiments where extensive empirical studies have been carried out (e.g. evaluated on >=5 classifiers and >=4 datasets).

### Links

https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
https://stackoverflow.com/questions/47793569/choosing-top-k-models-using-gridsearchcv-in-scikit-learn